# Import and Setup

## Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.preprocessing import StandardScaler

## Path and Data

In [2]:
path = r'C:\Users\rubix\Downloads\Machine Learning 1'

In [3]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-weather-prediction-dataset-processed.csv'), index_col = False)

# Checks Before Scaling

In [4]:
# Define what a quick check does

def quick_data_check(df, n_top_values=5):
    print("=== Shape ===")
    print(df.shape)
    print("\n=== Data Types ===")
    print(df.dtypes)
    
    print("\n=== First 5 Rows ===")
    print(df.head())
    
    print("\n=== Missing Values ===")
    print(df.isnull().sum())
    
    print("\n=== Basic Stats (Numeric Columns) ===")
    print(df.describe())
    
    print("\n=== Extended Numeric Summary ===")
    print(df.select_dtypes(include='number').agg(['mean', 'median', 'min', 'max']))
    
    print("\n=== Constant Columns ===")
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    print(constant_cols if constant_cols else "None")
    
    print("\n=== Mixed Data Types Check ===")
    for column in df.columns:
        inferred_type = pd.api.types.infer_dtype(df[column])
        print(f"Column '{column}': {inferred_type}")
        if inferred_type in ['mixed', 'mixed-integer']:
            print(f"  --> Column '{column}' has mixed data types.")  # Fixed indentation here
    
    print("\n=== Duplicate Row Counts ===")
    print(df.duplicated().value_counts())
    
    print("\n=== Top Values for Categorical Columns ===")
    for col in df.select_dtypes(include='object'):
        print(f"\nColumn: {col}")
        print(df[col].value_counts(dropna=False).head(n_top_values))
    
    print("\n=== Correlation Matrix (Numeric Only) ===")
    print(df.corr(numeric_only=True))

# Use quick_data_check(df) to call data

In [5]:
quick_data_check(df)

=== Shape ===
(22950, 170)

=== Data Types ===
DATE                     int64
MONTH                    int64
BASEL_cloud_cover        int64
BASEL_wind_speed       float64
BASEL_humidity         float64
                        ...   
VALENTIA_snow_depth      int64
VALENTIA_sunshine      float64
VALENTIA_temp_mean     float64
VALENTIA_temp_min      float64
VALENTIA_temp_max      float64
Length: 170, dtype: object

=== First 5 Rows ===
       DATE  MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0  19600101      1                  7               2.1            0.85   
1  19600102      1                  6               2.1            0.84   
2  19600103      1                  8               2.1            0.90   
3  19600104      1                  3               2.1            0.92   
4  19600105      1                  6               2.1            0.95   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0           1.018                    0.32     

## Summary of quick_data_check(df)

Nothing too crazy at first glance. I could maybe make a case for the date to be NOT an int64 type, but I think it's fine at the moment with the preferred YYYYMMDD format. 

Obviously I can't see literally all 170 columns at once using this method, but at least I get the names. It might be worth checking to see if all locations have the same number of columns. 

The correlation matrix is also kind of funny. I'm sure the correlations are going to be silly, but maybe a heatmap to see it better? Might be better after scaling so let's do that first.

# Scaling

## Separate Data

I don't seen any object/string types so I shouldn't have to limit to numerical only, but we'll check anyway. Should be 22950, 170 when done.

In [6]:
df_num = df.select_dtypes(include=np.number)

In [7]:
df_num.shape

(22950, 170)

Passed, awesome. Since I don't need that, I'll go ahead and delete it.

In [8]:
del df_num

That being said, we may have to return to the standard format for the DATE and MONTH columns eventually for practical displaying of data. I'll skip it for now.

## Scaling

In [9]:
scaler = StandardScaler()
# Use the column names from the original dataframe
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

## Checks

In [10]:
df_scaled.head()

DATE     MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0 -1.707663 -1.599964           0.660514          -0.02793        0.826097   
1 -1.707657 -1.599964           0.244897          -0.02793        0.735760   
2 -1.707652 -1.599964           1.076130          -0.02793        1.277781   
3 -1.707646 -1.599964          -1.001953          -0.02793        1.458455   
4 -1.707641 -1.599964           0.244897          -0.02793        1.729466   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0       -0.001949               -1.101066            -0.265148   
1       -0.001949               -1.058108             1.658760   
2       -0.001949               -1.251420             0.155707   
3       -0.001949               -0.821838            -0.445514   
4       -0.001949               -0.746661            -0.164944   

   BASEL_snow_depth  BASEL_sunshine  ...  VALENTIA_cloud_cover  \
0         -0.179228       -0.902918  ...             -0.443701   
1         -0.179228       -0.810126  ...              0.783085   
2         -0.179228       -1.065304  ...              0.783085   
3         -0.179228       -0.114186  ...              0.783085   
4         -0.179228        0.187388  ...             -1.670486   

   VALENTIA_humidity  VALENTIA_pressure  VALENTIA_global_radiation  \
0           0.761754          -1.299744                  -0.806427   
1           1.183580          -1.262455                  -1.042055   
2           1.183580          -0.432779                  -1.136306   
3           0.480538           0.387574                  -1.183432   
4          -0.363113           1.729970                  -0.794645   

   VALENTIA_precipitation  VALENTIA_snow_depth  VALENTIA_sunshine  \
0               -0.088407            -0.024706           0.372147   
1                0.503361            -0.024706          -0.829285   
2               -0.396127            -0.024706          -1.009500   
3                0.669056            -0.024706          -1.039536   
4               -0.490810            -0.024706           0.672505   

   VALENTIA_temp_mean  VALENTIA_temp_min  VALENTIA_temp_max  
0           -0.668215          -0.519743          -0.752237  
1           -0.548046          -0.629054          -0.407141  
2           -0.067372           0.054135          -0.177078  
3           -0.998679          -0.164486          -0.838511  
4           -1.509396          -1.339569          -1.471186  

[5 rows x 170 columns]

In [11]:
quick_data_check(df_scaled)

=== Shape ===
(22950, 170)

=== Data Types ===
DATE                   float64
MONTH                  float64
BASEL_cloud_cover      float64
BASEL_wind_speed       float64
BASEL_humidity         float64
                        ...   
VALENTIA_snow_depth    float64
VALENTIA_sunshine      float64
VALENTIA_temp_mean     float64
VALENTIA_temp_min      float64
VALENTIA_temp_max      float64
Length: 170, dtype: object

=== First 5 Rows ===
       DATE     MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0 -1.707663 -1.599964           0.660514          -0.02793        0.826097   
1 -1.707657 -1.599964           0.244897          -0.02793        0.735760   
2 -1.707652 -1.599964           1.076130          -0.02793        1.277781   
3 -1.707646 -1.599964          -1.001953          -0.02793        1.458455   
4 -1.707641 -1.599964           0.244897          -0.02793        1.729466   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0       -0.001949           

## Summary of Scaling and Checks

Everything seems to be in order so far. No lost data and all of it seems to have a standard deviation of 1.